## WELCOME TO WEEK 6

The Epic Finale Week

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK ❤️❤️❤️

### Please note

There may be changes here from the video as I'm always making updates!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To my Windows PC people - an important announcement</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

In [1]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace , set_trace_processors
from agents.mcp import MCPServerStdio
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel
from agents.tracing.processors import ConsoleSpanExporter, BatchTraceProcessor, default_processor
from agents.tracing.processor_interface import TracingExporter
from agents.tracing.spans import Span
from agents.tracing.traces import Trace
import os

In [2]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

### Special note - in case you updated the OpenAI Agents framework:

OpenAI Agents SDK recently upgraded their SDK and made a breaking change (thank you, OpenAI 😂). I haven't yet updated the version in this repo to the latest version. If you've upgraded the package yourself, then you might get an error about missing positional arguments. If you get that error, you'll need to replace `await server.list_tools()` with `await server.session.list_tools()` and change the last line from `fetch_tools` to `fetch_tools.tools`. And you'll need to make this change every time you get this error! The joys of working at the bleeding edge..

In [3]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

## Extra installation step - if you don't have Node and Playwright on your computer

The next MCP tool uses node (the Javascript Server), and it needs you to have the command 'npx' installed on your computer.

Extra setup steps are here:

[Node and Playwright setup](../setup/SETUP-node.md)

## And now 2 more MCP servers - this time Javascript based, with node

### If the next cell hangs, see the important troubleshooting at the end of the SETUP-node instructions linked above.

In [4]:

playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_messages', title=None, desc

In [5]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', title=None, description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='read_text_file', title=None, description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates 

### And now.. bring on the Agent with Tools!

In [6]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""
#Custom trace processor
class CustomConsoleSpanExporter(TracingExporter):
    def export(self, items: list[Trace | Span]):
        for item in items:
            if isinstance(item, Trace):
                print(f"[Trace] ID: {item.trace_id} | Name: {item.name}")
            elif item.span_data.type == "generation":
                usage = item.span_data.usage or {}
                model = item.span_data.model
                user_input = item.span_data.input or []
                output = item.span_data.output or []

                print("🧠 Model Used:", model)
                print("📥 Input Tokens:", usage.get("input_tokens", "N/A"))
                print("📤 Output Tokens:", usage.get("output_tokens", "N/A"))

                if user_input:
                    print("🙋 User Asked:", user_input[-1].get("content", "N/A"))
                if output:
                    print("🤖 Bot Replied:", output[0].get("content", "N/A"))
                    
BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_NAME='gemini-2.0-flash'

gemini_client = AsyncOpenAI(base_url=BASE_URL, api_key=os.getenv("GOOGLE_API_KEY"))
custom_model = OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=gemini_client) 

#Custom trace processor
exporter = CustomConsoleSpanExporter()
processor = BatchTraceProcessor(exporter)
set_trace_processors([processor])

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model=custom_model,
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for My Quang, then summarize it in markdown to My_Quang.md")
            print(result.final_output)


[Trace] ID: trace_a29e562c93f349209d8bd0ed3b1dd8f5 | Name: investigate
🧠 Model Used: gemini-2.0-flash
📥 Input Tokens: 1125
📤 Output Tokens: 24
🙋 User Asked: Find a great recipe for My Quang, then summarize it in markdown to My_Quang.md
🤖 Bot Replied: None
🧠 Model Used: gemini-2.0-flash
📥 Input Tokens: 19187
📤 Output Tokens: 60
🙋 User Asked: {"type":"text","text":"### Ran Playwright code\n```js\nawait page.goto('https://www.google.com/search?q=best+my+quang+recipe');\n```\n\n### New console messages\n- [WARNING] Failed to execute 'postMessage' on 'DOMWindow': The target origin provided ('https://www.y...\n\n### Page state\n- Page URL: https://www.google.com/search?q=best+my+quang+recipe\n- Page Title: best my quang recipe - Tìm trên Google\n- Page Snapshot:\n```yaml\n- generic [active] [ref=e1]:\n  - heading \"Liên kết hỗ trợ truy cập\" [level=1] [ref=e2]\n  - generic:\n    - generic:\n      - link \"Bỏ qua để đến phần nội dung chính\" [ref=e3] [cursor=pointer]\n      - link \"Hỗ trợ tr

### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



